In [2]:
%cd /home/parthgandhi/TradeBot

/home/parthgandhi/TradeBot


In [3]:
import polars as pl
import polars.selectors as cs
from src.config.storage_layout import StorageLayout
from src.config.market import Market
from src.config.exchange import Exchange

In [4]:
end_date = "2025-12-26"

In [6]:
classify_df = pl.scan_csv("/home/parthgandhi/Downloads/charts_maze_scanner.csv").rename(
    {
        "Stock Name": "symbol",
        "RS Rating": "rs_rating",
        "Basic Industry": "basic_industry",
        "Market Cap(Cr.)": "market_cap_cr",
        "1 Month Returns(%)": "1_mo_rtr_pct",
        "3 Month Returns(%)": "3_mo_rtr_pct",
        "% from 52W High": "pct_from_52w_high",
    }
)

In [18]:
filters_path = StorageLayout.filters_dir(
    run_date=end_date, market=Market.INDIA_EQUITIES, exchange=Exchange.NSE
)

basic_filter = (
    pl.scan_csv(filters_path / "basic_filter.csv")
    .with_columns(
        pl.col("timestamp")
        .str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S%.f")
        .cast(pl.Date)
        .alias("timestamp")
    )
    .select("timestamp", "symbol")
)

In [19]:
df_list = []
for filter_type in ["sma_200", "adr", "pullback", "reversal", "vcp"]:
    df = (
        pl.scan_csv(filters_path / f"{filter_type}_filter.csv")
        .with_columns(pl.lit(True).alias(f"{filter_type}_filter_flag"))
        .select("symbol", f"{filter_type}_filter_flag")
    )
    df_list.append(df)

In [20]:
res = (
    basic_filter.join(df_list[0], on="symbol", how="left")
    .join(df_list[1], on="symbol", how="left")
    .join(df_list[2], on="symbol", how="left")
    .join(df_list[3], on="symbol", how="left")
    .join(df_list[4], on="symbol", how="left")
    .with_columns(cs.ends_with("flag").fill_null(False))
    .join(classify_df, on="symbol", how="left")
    .collect()
)

In [21]:
print(f"Before RS filter: {res.shape}")
res.write_csv(filters_path / "overall_filter_result.csv")
res = res.filter(pl.col("rs_rating") >= 70)
print(f"After RS filter: {res.shape}")

Before RS filter: (368, 13)
After RS filter: (253, 13)


In [22]:
industry_analysis = (
    res.lazy()
    .group_by("basic_industry")
    .agg(
        [pl.col("symbol").count().alias("symbols_count")]
        + [
            (
                (pl.col(f"{i}_mo_rtr_pct") * pl.col("market_cap_cr")).sum()
                / pl.col("market_cap_cr").sum()
            )
            .round(2)
            .alias(f"{i}_mo_avg_weight_rtr_pct")
            for i in [1, 3]
        ]
        + [
            (
                (pl.col("rs_rating") * pl.col("market_cap_cr")).sum()
                / pl.col("market_cap_cr").sum()
            )
            .round(2)
            .alias(f"rs_rating_avg_weight")
        ]
    )
    .fill_nan(None)
    .sort("1_mo_avg_weight_rtr_pct", descending=True)
    .with_columns(
        pl.mean_horizontal(cs.exclude("basic_industry", "symbols_count"))
        .round(2)
        .alias("industry_score"),
        (pl.col("1_mo_avg_weight_rtr_pct") * 100 / pl.col("3_mo_avg_weight_rtr_pct"))
        .round()
        .alias("1_by_3_rtr"),
    )
    .collect()
)

In [23]:
industry_analysis.write_csv(filters_path / "industry_analysis.csv")

# RS >= 70 & Possible Upcoming Centers

In [24]:
industry_analysis.filter(
    (pl.col("rs_rating_avg_weight") >= 70)
    & (pl.col("1_mo_avg_weight_rtr_pct") >= pl.col("3_mo_avg_weight_rtr_pct"))
).sort("industry_score", descending=True)

basic_industry,symbols_count,1_mo_avg_weight_rtr_pct,3_mo_avg_weight_rtr_pct,rs_rating_avg_weight,industry_score,1_by_3_rtr
str,u32,f64,f64,f64,f64,f64
"""Media & Entertainment""",1,43.0,33.5,99.0,58.5,128.0
"""Packaging""",1,17.2,6.3,93.0,38.83,273.0
"""Railways""",1,19.9,14.8,73.0,35.9,134.0
"""Oil & Gas Drilling""",1,15.6,8.8,76.0,33.47,177.0


In [25]:
industry_analysis.filter((pl.col("rs_rating_avg_weight") >= 70)).sort(
    "industry_score", descending=True
).filter(pl.col("symbols_count") > 1).to_pandas()

,basic_industry,symbols_count,1_mo_avg_weight_rtr_pct,3_mo_avg_weight_rtr_pct,rs_rating_avg_weight,industry_score,1_by_3_rtr
0,Mining/Minerals,8,25.44,35.95,92.61,51.33,71.0
1,NBFC,10,7.77,40.32,95.30,47.80,19.0
2,Tyres & Rubber Products,3,8.16,35.67,93.29,45.71,23.0
3,Construction Products Miscallaneous,3,9.42,32.32,88.84,43.53,29.0
4,Diversified Commercial Services,3,10.17,30.77,85.78,42.24,33.0
5,Metal Fabrication,6,10.98,24.89,90.42,42.10,44.0
6,Finance & Investment,5,-1.06,25.98,96.36,40.43,-4.0
7,Financial Services-Specialty,4,-1.08,27.51,93.86,40.10,-4.0
8,Medical-Diversified,5,8.23,22.76,88.91,39.97,36.0
9,Agro Products,4,0.22,27.17,86.30,37.90,1.0
